In [13]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchsummary import summary

from PIL import Image

import config

In [16]:
model = models.resnet18(pretrained=True)

In [17]:
modules=list(model.children())[:-1]
model=nn.Sequential(*modules)
for param in model.parameters():
    param.requires_grad = False
model.to(config.device)

summary(model, input_size=(3, 214, 214))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 107, 107]           9,408
       BatchNorm2d-2         [-1, 64, 107, 107]             128
              ReLU-3         [-1, 64, 107, 107]               0
         MaxPool2d-4           [-1, 64, 54, 54]               0
            Conv2d-5           [-1, 64, 54, 54]          36,864
       BatchNorm2d-6           [-1, 64, 54, 54]             128
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]          36,864
       BatchNorm2d-9           [-1, 64, 54, 54]             128
             ReLU-10           [-1, 64, 54, 54]               0
       BasicBlock-11           [-1, 64, 54, 54]               0
           Conv2d-12           [-1, 64, 54, 54]          36,864
      BatchNorm2d-13           [-1, 64, 54, 54]             128
             ReLU-14           [-1, 64,

In [7]:
scalar = transforms.Scale((224,224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [8]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)

    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    
    # 6. Run the model on our transformed image
    model(t_img)
    
    # 7. Detach our copy function from the layer
    h.remove()
    
    # 8. Return the feature vector
    return my_embedding